In [2]:

import glob
import json
import csv
import os
import random


In [3]:
! pip install pandas aiofiles

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [ ]:
import asyncio
import aiofiles
import pandas as pd


semaphore = asyncio.Semaphore(50)
async def read_json_file(file):
    print (file)
    """Read JSON files and extract required fields."""
    async with semaphore:
        async with aiofiles.open(file, "r", encoding="utf-8") as f:
            try:
                data = json.loads(await f.read())
                try:
                    return data["results"]

                except KeyError:
                    print(f"⚠️ Missing 'results' field in {file}")
                    return None

            except json.JSONDecodeError:
                print(f"❌ Error parsing {file}")
                return None
            
async def write_json_file(file, data):
    """Write JSON files asynchronously."""
    async with semaphore:
        async with aiofiles.open(file, "w", encoding="utf-8") as f:
            await f.write(json.dumps(data, indent=4, ensure_ascii=False))
            print(f"✅ {file} written.")
            return True

DATATYPE="TAVG"
DATASET="GSOM"  
DATE="2023-03-14_2025-03-14"
OUTPUT_FILE_JSON=f"{DATATYPE}_{DATASET}_{DATE}.json"
OUTPUT_FILE_CSV=f"{DATATYPE}_{DATASET}_{DATE}.csv"
# INPUT_DIR=f"./data/{DATE}/{DATASET}/{DATATYPE}/"
INPUT_DIR="./api_responses/datasets/"
OUTPUT_FILE_JSON="datasets_merged.json"
OUTPUT_FILE_CSV="datasets_merged.csv"
async def main():
    all_json_files = glob.glob(f"{INPUT_DIR}*.json")
    print("{} JSON files found.".format(len(all_json_files)))
    tasks=[read_json_file(file) for file in all_json_files]
    results = await asyncio.gather(*tasks)
    flattened_list = [item for sublist in results if sublist is not None for item in sublist]
    json_dict={item['id']:item['name'] for item in flattened_list}
 
    await write_json_file(OUTPUT_FILE_JSON, json_dict)
    
await main()


2 JSON files found.
./api_responses/datasets/datasets_offset_0.json
./api_responses/datasets/datasets_offset_10.json
{'GHCND': 'Daily Summaries', 'GSOM': 'Global Summary of the Month', 'GSOY': 'Global Summary of the Year', 'NEXRAD2': 'Weather Radar (Level II)', 'NEXRAD3': 'Weather Radar (Level III)', 'NORMAL_ANN': 'Normals Annual/Seasonal', 'NORMAL_DLY': 'Normals Daily', 'NORMAL_HLY': 'Normals Hourly', 'NORMAL_MLY': 'Normals Monthly', 'PRECIP_15': 'Precipitation 15 Minute', 'PRECIP_HLY': 'Precipitation Hourly'}


In [27]:
df=pd.read_json(OUTPUT_FILE_JSON)
# df.rename(columns={"date": "date", "datatype": "datatype", "station": "station_id", "attributes": "attributes", "value": "value"}, inplace=True)
# df['dataset_name']="GSOM"
# sorted_columns = ["dataset_name",  "station_id","date","datatype","attributes", "value"]
# df = df[sorted_columns]
df.head()
print (len(df))
# print (len(df))
# df.drop_duplicates(subset=["station_id","date","datatype"], keep="first", inplace=True)
# print (len(df))

1567


In [28]:
df.to_csv(OUTPUT_FILE_CSV, index=False)
